In [16]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
from dataset import InputFeatures, load_and_cache_examples
import dataset
import functionsAval as f

In [29]:
f = reload(f)
reload(dataset)
reload(eval)


<module 'eval' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\eval.py'>

In [124]:
# em numero de frases
#BATCH=30
#BATCH=100 
BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<=BATCH}
#print(dicSentences_new_test[0])
print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))


sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])



BATCH: 800
Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Nega', 0, 422], ['sincope', 1, 427], ['.', 2, 434]], [['sincope', [1], 'Problema']]]
numero de sentencas no total: 506


In [125]:
tipos_entidade = ['Anatomia','Problema','Tratamento','Teste']
#tipos_entidade = ['Anatomia']
all_predictions = list()
for tipo_entidade in tipos_entidade:
    tags, tokens = f.predictBERTNER_IO(sentences, tipo_entidade)
    dic_predictions = f.getDicPredictions(tags, tokens)
    print(dic_predictions[0])
    print('len(dic_predictions):', len(dic_predictions))
    all_predictions.append(dic_predictions)
    #f.save_obj('dic_predictions_io_bkp', dic_predictions)
#dic_predictions = load_obj('dic_predictions_io_bkp')


idx2tag: {0: 'O', 1: 'Anatomia', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
idx2tag: {0: 'Problema', 1: 'O', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
idx2tag: {0: 'O', 1: 'Tratamento', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
idx2tag: {0: 'O', 1: 'Teste', 2: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506


In [130]:
len(all_predictions)

4

In [134]:
len(all_predictions[0])

506

In [135]:
all_predictions[0][9]

[[['Abd', 0],
  ['globoso', 1],
  [',', 2],
  ['flacido', 3],
  [',', 4],
  ['indolor', 5],
  ['a', 6],
  ['palpacao', 7],
  [',', 8],
  ['sem', 9],
  ['VCM', 10],
  ['.', 11]],
 [[['Abd'], [0], 'Anatomia']]]

In [136]:
f.save_obj('all_predictions_results_binarios_'+str(BATCH), all_predictions)
#all_predictions = f.load_obj('all_predictions_results_binarios_batch_'+str(BATCH))

In [137]:
# juntar all_predictions
dic_predictions = {}
for num, prediction in enumerate(all_predictions):
    if num==0:
        dic_predictions = prediction.copy()
        continue
    for key, value in prediction.items():
        entidadesJaEstavam=dic_predictions[key][1]
        tokens=dic_predictions[key][0]
        lista_entidade = [e for e in entidadesJaEstavam]
        #print('lista_entidade:', lista_entidade)
        entidades = value[1].copy()
        if len(entidades)>0:
            #print(entidades)
            for entidade in entidades:
                lista_entidade.append(entidade)
            dic_predictions[key]=[tokens,lista_entidade]


In [138]:
for key, value in dic_predictions.items():
    print('key:',key)
    print(dic_predictions[key])
    if key>3:
        break

key: 0
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
key: 1
[[['Em', 0], ['acompanhamento', 1], ['no', 2], ['ambualtorio', 3], ['há', 4], ['5', 5], ['anos', 6], ['por', 7], ['FA', 8], [',', 9], ['uso', 10], ['de', 11], ['marevan', 12], ['5mg', 13], ['1', 14], ['x', 15], ['ao', 16], ['dia', 17], ['.', 18]], [[['FA'], [8], 'Problema'], [['marevan', '5mg'], [12, 13], 'Tratamento']]]
key: 2
[[['Comorbidades', 0], [':', 1], ['DM', 2], ['há', 3], ['10', 4], ['anos', 5], ['em', 6], ['uso', 7], ['de', 8], ['metformina', 9], ['850mg', 10], ['3', 11], ['cp', 12], ['/', 13], ['dia', 14], [',', 15], ['acarbose', 16], ['1', 17], ['cp', 18], ['/', 19], ['dia', 20], ['e', 21], ['glicazida', 22], ['60mg', 23], ['2', 24], ['cp', 25], ['/', 26], ['dia', 27], ['e', 28], ['insulina', 29], ['(', 30], ['24', 31], ['-', 32], ['0', 33], ['-', 34], ['24', 35], [')', 36], ['.', 37]], [[['Comorbidades'], [0], 'Problema'], [['DM'], [2], 'Problema'], [['metformina', '850mg'], [9, 10], 'Tratament

In [146]:
f = reload(f)
reload(dataset)
reload(eval)


<module 'eval' from 'C:\\Users\\lisat\\OneDrive\\jupyter notebook\\spanclassification\\avaliacao\\eval.py'>

In [140]:
dic_predictions[1][1]

[[['FA'], [8], 'Problema'], [['marevan', '5mg'], [12, 13], 'Tratamento']]

In [143]:
print(len(dicSentences_new_test))
print(len(dic_predictions))

506
506


In [147]:
region_true_list, region_pred_list = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)


In [148]:
region_pred_list[:4]

['Problema', 'Tratamento', 'Problema', 'Problema']

In [149]:
region_true_list[:4]

['Problema', 'Tratamento', 'Problema', 'Problema']

In [150]:
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))




len(region_true_list): 1190
len(region_pred_list): 1190
-----Avaliando só modelo de NER:-----
              precision    recall  f1-score   support

    Anatomia   0.786730  0.846939  0.815725       196
           O   0.000000  0.000000  0.000000       199
    Problema   0.760753  0.837278  0.797183       338
       Teste   0.867769  0.864198  0.865979       243
  Tratamento   0.810185  0.817757  0.813953       214

    accuracy                       0.700840      1190
   macro avg   0.645087  0.673234  0.658568      1190
weighted avg   0.668555  0.700840  0.683991      1190

[[166  27   3   0   0]
 [ 44   0  83  32  40]
 [  0  55 283   0   0]
 [  1  31   0 210   1]
 [  0  36   3   0 175]]


In [ ]:

print('-----Avaliando só com modelo de Spam:-----')
model = f.getModel()
dicPosTagger = f.getDicPosTagger()
lista_postaggers_entidades = f.getListaPostaggerEntidades(dic_predictions, dicPosTagger)

#print('-CC' in lista_postaggers_entidades) # -> só tem 2, um é em maiusculo, excluir

# para gerar arquivo de predicoes (com as tags <e1>)
#combinacaoEntidadesAll = f.getCombinacaoEntidadesAll(dic_predictions, dicPosTagger, lista_postaggers_entidades, True)
combinacaoEntidadesAll = f.getCombinacaoEntidadesAllPosProc(dic_predictions, dicPosTagger, lista_postaggers_entidades, True)
#print(combinacaoEntidadesAll[3])
f.gravaArquivoPredict(combinacaoEntidadesAll)

In [228]:
# chamar predict                
pred_region_labels = f.predictSpan(model)
print('len(pred_region_labels):', len(pred_region_labels))

f.save_obj('combinacaoEntidadesAll_io_bkp', combinacaoEntidadesAll)
#combinacaoEntidadesAll = load_obj('combinacaoEntidadesAll_io_bkp')

# combinacaoEntidadesAll_pred = predicoes do SpanClassification no formato para comparar com gabarito
combinacaoEntidadesAll_pred = f.getCombinacaoEntidadesAll_pred(combinacaoEntidadesAll, pred_region_labels)

dic_predictions_span = f.getDicPredictionsSpan(combinacaoEntidadesAll_pred, dic_predictions)

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_span, BATCH)


# tbm sem o postagger pra ver

print('-----Avaliando os dois modelos juntos:-----')

# dic_predictions_all = mesclando o resultado dos dois modelos (dic_predictions + combinacaoEntidadesAll_pred)
dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)
f.save_obj('dic_predictions_all_io_bkp', dic_predictions_all)
#dic_predictions_all = load_obj('dic_predictions_all_io_bkp')

LOOKING AT ./data\genia.predict
lendo arquivo:  ./data\genia.predict
len(pred_region_labels): 770
num: 770
25
26
29
32
42
45
46
65
numErro1: 173
numErro2: 152
              precision    recall  f1-score   support

   CELL_LINE   0.368421  0.411765  0.388889        34
   CELL_TYPE   0.950000  0.633333  0.760000        30
         DNA   0.523256  0.384615  0.443350       117
           O   0.000000  0.000000  0.000000       152
     PROTEIN   0.375000  0.418605  0.395604       172
         RNA   0.000000  0.000000  0.000000         4

    accuracy                       0.294695       509
   macro avg   0.369446  0.308053  0.331307       509
weighted avg   0.327598  0.294695  0.306362       509

-----Avaliando os dois modelos juntos:-----
numAcrescentou: 32


In [231]:
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

numErro1: 125
numErro2: 104
              precision    recall  f1-score   support

   CELL_LINE   0.576923  0.441176  0.500000        34
   CELL_TYPE   0.785714  0.733333  0.758621        30
         DNA   0.715596  0.666667  0.690265       117
           O   0.000000  0.000000  0.000000       104
     PROTEIN   0.581395  0.581395  0.581395       172
         RNA   0.000000  0.000000  0.000000         4

    accuracy                       0.466377       461
   macro avg   0.443272  0.403762  0.421714       461
weighted avg   0.492216  0.466377  0.478351       461



In [233]:
dic_predictions = f.load_obj('dic_predictions_io_bkp')
combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_io_bkp')
dic_predictions_all = f.load_obj('dic_predictions_all_io_bkp')

In [234]:
dic_predictions_span

{0: [[['Characterization', 0],
   ['of', 1],
   ['HB24', 2],
   ['expression', 3],
   ['in', 4],
   ['lymphoid', 5],
   ['and', 6],
   ['select', 7],
   ['developing', 8],
   ['tissues', 9],
   ['was', 10],
   ['performed', 11],
   ['by', 12],
   ['in', 13],
   ['situ', 14],
   ['hybridization', 15],
   ['.', 16]],
  [[['HB24'], [2], 'CELL_LINE']]],
 1: [[['Positive', 0],
   ['hybridization', 1],
   ['was', 2],
   ['found', 3],
   ['in', 4],
   ['thymus', 5],
   [',', 6],
   ['tonsil', 7],
   [',', 8],
   ['bone', 9],
   ['marrow', 10],
   [',', 11],
   ['developing', 12],
   ['vessels', 13],
   [',', 14],
   ['and', 15],
   ['in', 16],
   ['fetal', 17],
   ['brain', 18],
   ['.', 19]],
  []],
 2: [[['HB24', 0],
   ['is', 1],
   ['likely', 2],
   ['to', 3],
   ['have', 4],
   ['an', 5],
   ['important', 6],
   ['role', 7],
   ['in', 8],
   ['lymphocytes', 9],
   ['as', 10],
   ['well', 11],
   ['as', 12],
   ['in', 13],
   ['certain', 14],
   ['developing', 15],
   ['tissues', 16],
   

In [235]:
dic_predictions

{0: [[['Characterization', 0],
   ['of', 1],
   ['HB24', 2],
   ['expression', 3],
   ['in', 4],
   ['lymphoid', 5],
   ['and', 6],
   ['select', 7],
   ['developing', 8],
   ['tissues', 9],
   ['was', 10],
   ['performed', 11],
   ['by', 12],
   ['in', 13],
   ['situ', 14],
   ['hybridization', 15],
   ['.', 16]],
  [[['HB24'], [2], 'PROTEIN']]],
 1: [[['Positive', 0],
   ['hybridization', 1],
   ['was', 2],
   ['found', 3],
   ['in', 4],
   ['thymus', 5],
   [',', 6],
   ['tonsil', 7],
   [',', 8],
   ['bone', 9],
   ['marrow', 10],
   [',', 11],
   ['developing', 12],
   ['vessels', 13],
   [',', 14],
   ['and', 15],
   ['in', 16],
   ['fetal', 17],
   ['brain', 18],
   ['.', 19]],
  []],
 2: [[['HB24', 0],
   ['is', 1],
   ['likely', 2],
   ['to', 3],
   ['have', 4],
   ['an', 5],
   ['important', 6],
   ['role', 7],
   ['in', 8],
   ['lymphocytes', 9],
   ['as', 10],
   ['well', 11],
   ['as', 12],
   ['in', 13],
   ['certain', 14],
   ['developing', 15],
   ['tissues', 16],
   ['

In [236]:
dicSentences_new_test

{0: [[['Characterization', 0],
   ['of', 1],
   ['HB24', 2],
   ['expression', 3],
   ['in', 4],
   ['lymphoid', 5],
   ['and', 6],
   ['select', 7],
   ['developing', 8],
   ['tissues', 9],
   ['was', 10],
   ['performed', 11],
   ['by', 12],
   ['in', 13],
   ['situ', 14],
   ['hybridization', 15],
   ['.', 16]],
  [[['HB24'], [2], 'DNA']]],
 1: [[['Positive', 0],
   ['hybridization', 1],
   ['was', 2],
   ['found', 3],
   ['in', 4],
   ['thymus', 5],
   [',', 6],
   ['tonsil', 7],
   [',', 8],
   ['bone', 9],
   ['marrow', 10],
   [',', 11],
   ['developing', 12],
   ['vessels', 13],
   [',', 14],
   ['and', 15],
   ['in', 16],
   ['fetal', 17],
   ['brain', 18],
   ['.', 19]],
  []],
 2: [[['HB24', 0],
   ['is', 1],
   ['likely', 2],
   ['to', 3],
   ['have', 4],
   ['an', 5],
   ['important', 6],
   ['role', 7],
   ['in', 8],
   ['lymphocytes', 9],
   ['as', 10],
   ['well', 11],
   ['as', 12],
   ['in', 13],
   ['certain', 14],
   ['developing', 15],
   ['tissues', 16],
   ['.', 

In [237]:
dic_predictions_all

{0: [[['Characterization', 0],
   ['of', 1],
   ['HB24', 2],
   ['expression', 3],
   ['in', 4],
   ['lymphoid', 5],
   ['and', 6],
   ['select', 7],
   ['developing', 8],
   ['tissues', 9],
   ['was', 10],
   ['performed', 11],
   ['by', 12],
   ['in', 13],
   ['situ', 14],
   ['hybridization', 15],
   ['.', 16]],
  [[['HB24'], [2], 'PROTEIN']]],
 1: [[['Positive', 0],
   ['hybridization', 1],
   ['was', 2],
   ['found', 3],
   ['in', 4],
   ['thymus', 5],
   [',', 6],
   ['tonsil', 7],
   [',', 8],
   ['bone', 9],
   ['marrow', 10],
   [',', 11],
   ['developing', 12],
   ['vessels', 13],
   [',', 14],
   ['and', 15],
   ['in', 16],
   ['fetal', 17],
   ['brain', 18],
   ['.', 19]],
  []],
 2: [[['HB24', 0],
   ['is', 1],
   ['likely', 2],
   ['to', 3],
   ['have', 4],
   ['an', 5],
   ['important', 6],
   ['role', 7],
   ['in', 8],
   ['lymphocytes', 9],
   ['as', 10],
   ['well', 11],
   ['as', 12],
   ['in', 13],
   ['certain', 14],
   ['developing', 15],
   ['tissues', 16],
   ['